In [1]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal


In [2]:
import aiida
aiida.load_profile()


In [3]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


In [4]:
# nodebank settings

from tools.nodebank import NodeBank
pwd = os.getcwd()
print(pwd)
nodebank = NodeBank(pwd)


/home/max/Documents/aiida_cryspy/example/lammps_GaN_8_EA


In [5]:
import os
import numpy as np


from aiida.plugins import DataFactory
from aiida.orm import Code
from aiida.orm import Str, Dict, List, Int
from aiida.engine import calcfunction, WorkChain

from CrySPY.IO import read_input as rin
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')

ConfigparserData = DataFactory('cryspy.configparser')
PystructuredictData = DataFactory('cryspy.pystructuredict')
PandasFrameData = DataFactory('cryspy.dataframe')

SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


class initialize_WorkChain(WorkChain):
    """start.initialize()
    """

    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input("cryspy_in", valid_type=SinglefileData, help='cryspy_in. (temporary implementation)')

        spec.outline(
            cls.start_initialize,
            cls.make_aiidadata,
        )
        spec.output("init_struc", valid_type=PystructuredictData, help='initial structures')
        spec.output("opt_struc", valid_type=PystructuredictData, help='optimized structures')
        spec.output("rstl_data", valid_type=PandasFrameData, help='rst_data')
        spec.output("ea_info", valid_type=PandasFrameData, help='ea_info')
        spec.output("ea_origin", valid_type=PandasFrameData, help='ea_origin')
        spec.output("ea_id_gen", valid_type=Int, help='ea_origin')
        spec.output("ea_id_queueing", valid_type=List, help='ea_origin')
        spec.output("ea_id_running", valid_type=List, help='ea_origin')
        spec.output('stat', valid_type=ConfigparserData, help='configparser content')

    def start_initialize(self):
        from CrySPY.start import cryspy_init
        cryspy_init.initialize(self.inputs.cryspy_in)

    def make_aiidadata(self):
        """must handle
        EA_data.pkl  
        EA_id_data.pkl  
        init_struc_data.pkl  
        opt_struc_data.pkl  
        rslt_data.pkl
        """
        if False:
            @calcfunction
            def aiida_load_init_struc():
                from CrySPY.IO.pkl_data import load_init_struc
                struc_dict = load_init_struc()
                structures = {}
                for _i, value in struc_dict.items():
                    content = value.as_dict()
                    key = f'ID_{_i}'
                    structures[key] = content
                return Dict(dict=structures)
            self.out('init_struc', aiida_load_init_struc())
        else:
            @calcfunction
            def aiida_load_init_struc(): 
                from CrySPY.IO.pkl_data import load_init_struc
                struc_dict = load_init_struc()
                pystructuredict = PystructuredictData(struc_dict)
                return pystructuredict
            self.out('init_struc', aiida_load_init_struc())
            
        if False:
            @calcfunction
            def aiida_load_opt_struc():
                from CrySPY.IO.pkl_data import load_opt_struc
                struc_dict = load_opt_struc()
                structures = {}
                for _i, value in struc_dict.items():
                    content = value.as_dict()
                    key = f'ID_{_i}'
                    structures[key] = content
                return Dict(dict=structures)
            self.out('opt_struc', aiida_load_opt_struc())
        else:
            @calcfunction
            def aiida_load_opt_struc():
                from CrySPY.IO.pkl_data import load_opt_struc
                struc_dict = load_opt_struc()
                pystructuredict = PystructuredictData(struc_dict)
                return pystructuredict
            self.out('opt_struc', aiida_load_opt_struc())

        if False:
            @calcfunction
            def aiida_load_rslt():
                from CrySPY.IO.pkl_data import load_rslt
                rstl_data_dic = {}
                rstl_data = load_rslt()
                for key in rstl_data.columns:
                    rstl_data_dic[key] = rstl_data[key].values.tolist()
                return Dict(dict=rstl_data_dic)
            self.out('rstl_data', aiida_load_rslt())
        else:
            @calcfunction
            def aiida_load_rslt():
                from CrySPY.IO.pkl_data import load_rslt
                rstl_data = load_rslt()
                display(rstl_data)
                df_node = PandasFrameData(rstl_data)
                return df_node
            self.out('rstl_data', aiida_load_rslt())

        if False:
            @calcfunction
            def aiida_load_ea_info():
                from CrySPY.IO.pkl_data import load_ea_data
                ea_data = {}
                elite_struc, elite_fitness, ea_info, ea_origin = load_ea_data()
                ea_info_dic = {}
                for key in ea_info.columns:
                    ea_info_dic[key] = ea_info[key].values.tolist()
                return Dict(dict=ea_info_dic)
            @calcfunction
            def aiida_load_ea_origin():
                from CrySPY.IO.pkl_data import load_ea_data
                ea_data = {}
                elite_struc, elite_fitness, ea_info, ea_origin = load_ea_data()
                ea_origin_dic = {}
                for key in ea_origin.columns:
                    ea_origin_dic[key] = ea_origin[key].values.tolist()
                return Dict(dict=ea_origin_dic)
            self.out('ea_info', aiida_load_ea_info())
            self.out('ea_origin', aiida_load_ea_origin())            
        else:
            from CrySPY.IO.pkl_data import load_ea_data
            @calcfunction
            def aiida_load_ea_info():
                elite_struc, elite_fitness, ea_info, ea_origin = load_ea_data()
                df_ea_info_node = PandasFrameData(ea_info)
                return df_ea_info_node
            @calcfunction
            def aiida_load_ea_origin():
                elite_struc, elite_fitness, ea_info, ea_origin = load_ea_data()
                df_ea_origin_node = PandasFrameData(ea_origin)
                return df_ea_origin_node
            self.out('ea_info', aiida_load_ea_info())
            self.out('ea_origin', aiida_load_ea_origin()) 

        @calcfunction
        def aiida_load_ea_id_gen():
            from CrySPY.IO.pkl_data import load_ea_id
            gen, id_queueing, id_running = load_ea_id()
            return Int(gen)

        @calcfunction
        def aiida_load_ea_id_queueing():
            from CrySPY.IO.pkl_data import load_ea_id
            gen, id_queueing, id_running = load_ea_id()
            return List(list=id_queueing)

        @calcfunction
        def aiida_load_ea_id_running():
            from CrySPY.IO.pkl_data import load_ea_id
            gen, id_queueing, id_running = load_ea_id()
            return List(list=id_running)
        self.out('ea_id_gen', aiida_load_ea_id_gen())
        self.out('ea_id_queueing', aiida_load_ea_id_queueing())
        self.out('ea_id_running', aiida_load_ea_id_running())


In [6]:
pwd = os.getcwd()
cryspy_in = SinglefileData(os.path.join(pwd, 'cryspy.in0'))
inputs = {'cryspy_in': cryspy_in}
# initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
initial_result = run(initialize_WorkChain, **inputs)


2022/09/12 08:06:43
CrySPY 0.10.3
Start cryspy.py

Read input file, uuid: 82694dfc-e2bf-4204-8a2f-b5604df60592 (pk: 17690)


readin


Write input data in cryspy.out
Save input data in cryspy.stat


write_stat



# --------- Generate initial structures
Structure ID      0 was generated. Space group:   2 -->   2 P-1
Structure ID      1 was generated. Space group: 127 --> 123 P4/mmm
Structure ID      2 was generated. Space group: 212 --> 212 P4_332
Structure ID      3 was generated. Space group:  20 -->  20 C222_1
Structure ID      4 was generated. Space group: 149 --> 187 P-6m2
Structure ID      5 was generated. Space group:  34 -->  34 Pnn2
Structure ID      6 was generated. Space group: 100 -->  99 P4mm
Structure ID      7 was generated. Space group:  41 -->  41 Aea2


Compoisition [16 16] not compatible with symmetry 225 :spg = 225 retry.


Structure ID      8 was generated. Space group:  12 -->  12 C2/m
Structure ID      9 was generated. Space group: 182 --> 194 P6_3/mmc
Structure ID     10 was generated. Space group:  20 -->  20 C222_1
Structure ID     11 was generated. Space group:  26 -->  26 Pmc2_1
Structure ID     12 was generated. Space group:  62 -->  62 Pnma
Structure ID     13 was generated. Space group: 191 --> 191 P6/mmm
Structure ID     14 was generated. Space group:  92 -->  92 P4_12_12
Structure ID     15 was generated. Space group: 104 --> 107 I4mm
Structure ID     16 was generated. Space group:  84 --> 131 P4_2/mmc


Compoisition [16 16] not compatible with symmetry 228 :spg = 228 retry.


Structure ID     17 was generated. Space group: 149 --> 187 P-6m2
Structure ID     18 was generated. Space group: 140 --> 140 I4/mcm
Structure ID     19 was generated. Space group:  56 -->  59 Pmmn
Structure ID     20 was generated. Space group:  89 --> 123 P4/mmm
Structure ID     21 was generated. Space group:  98 -->  98 I4_122
Structure ID     22 was generated. Space group: 158 --> 156 P3m1
Structure ID     23 was generated. Space group:  91 -->  91 P4_122
Structure ID     24 was generated. Space group:  17 -->  17 P222_1
Structure ID     25 was generated. Space group: 188 --> 187 P-6m2
Structure ID     26 was generated. Space group:  93 --> 131 P4_2/mmc
Structure ID     27 was generated. Space group:  91 -->  91 P4_122
Structure ID     28 was generated. Space group: 165 --> 164 P-3m1
Structure ID     29 was generated. Space group:  58 -->  58 Pnnm


save_init_struc




save_opt_struc




save_rslt



# ---------- Initialize evolutionary algorithm
# ------ Generation 1
30 structures

Compoisition [4 4] not compatible with symmetry 223 :spg = 223 retry.
Compoisition [4 4] not compatible with symmetry 145 :spg = 145 retry.




save_ea_id




save_ea_data




save_rslt




write_stat




load_init_struc




load_opt_struc




load_rslt




,Gen,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt




load_ea_data




load_ea_data




load_ea_id




load_ea_id




load_ea_id




In [ ]:
# workflow結果表示
initial_result

In [ ]:
initial_result["init_struc"].pystructuredict

In [ ]:
import shutil
# cryspy.inは実行中に書き換えられる。
shutil.copy("cryspy.in0", "cryspy.in")


In [ ]:
init_struc_data = initial_result["init_struc"]
init_struc_data.pystructuredict


In [ ]:
initial_result

In [ ]:
# where is crpspy.stat file?

# restart

In [ ]:
'''
Restart CrySPY
'''

import os

from CrySPY import utility
from CrySPY.BO import bo_restart
from CrySPY.EA import ea_append
from CrySPY.gen_struc.random.random_generation import Rnd_struc_gen
from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal
from CrySPY.IO import io_stat, pkl_data
from CrySPY.IO import read_input as rin
from CrySPY.LAQA import laqa_restart
from CrySPY.RS import rs_restart


def restart(cryspy_in='cryspy.in', cryspy_stat='cryspy.stat',
            aiida=False,
            stat=None, init_struc_data=None):
    """
    under construction.
    
    
    cryspy_in can be SinglefileData.
    
    I can't understand the flow chart when SystemExit() occurs.
    
    """
    print('\n\n')
    print(utility.get_date())
    print(utility.get_version())
    print('Restart cryspy.py\n\n')
    
    if aiida:
        if stat is None:
            raise ValueError('stat must be given.')
        if init_struc_data is None:
            raise ValueError('init_struc_data must be given.')

    # ---------- read stat
    if not aiida:
        stat = io_stat.stat_read(cryspy_stat)

    # ---------- read input and check the change
    rin.readin(cryspy_in)
    rin.diffinstat(stat)

    # ---------- load init_struc_data for appending structures
    if not aiida:
        init_struc_data = pkl_data.load_init_struc()

    # ---------- append structures
    if len(init_struc_data) < rin.tot_struc:
        prev_nstruc = len(init_struc_data)
        init_struc_data = append_struc(init_struc_data)
        # ------ RS
        if rin.algo == 'RS':
            rs_restart.restart(stat, prev_nstruc)
        # ------ BO
        if rin.algo == 'BO':
            bo_restart.restart(init_struc_data, prev_nstruc)
        # ------ LAQA
        if rin.algo == 'LAQA':
            laqa_restart.restart(stat, prev_nstruc)
        os.remove('lock_cryspy')
        raise SystemExit()

    elif rin.tot_struc < len(init_struc_data):
        raise ValueError('tot_struc < len(init_struc_data)')

    # ---------- append structures by EA (option)
    if rin.append_struc_ea:
        prev_nstruc = len(init_struc_data)
        init_struc_data = ea_append.append_struc(stat, init_struc_data, aiida=aiida)
        # ------ RS
        if rin.algo == 'RS':
            rs_restart.restart(stat, prev_nstruc)
        # ------ BO
        if rin.algo == 'BO':
            bo_restart.restart(init_struc_data, prev_nstruc)
        # ------ LAQA
        if rin.algo == 'LAQA':
            laqa_restart.restart(stat, prev_nstruc)
        os.remove('lock_cryspy')
        raise SystemExit()

    # ---------- return
    return stat, init_struc_data


def append_struc(init_struc_data, aiida=False):
    # ---------- append initial structures
    print('\n# ---------- Append structures')
    with open('cryspy.out', 'a') as fout:
        fout.write('\n# ---------- Append structures\n')
    id_offset = len(init_struc_data)
    nstruc = rin.tot_struc - id_offset

    # ---------- pyxtal
    if not (rin.spgnum == 0 or rin.use_find_wy):
        rsgx = Rnd_struc_gen_pyxtal(natot=rin.natot, atype=rin.atype,
                                    nat=rin.nat, vol_factor=rin.vol_factor,
                                    vol_mu=rin.vol_mu, vol_sigma=rin.vol_sigma,
                                    mindist=rin.mindist,
                                    spgnum=rin.spgnum, symprec=rin.symprec)
        # ------ crystal
        if rin.struc_mode == 'crystal':
            rsgx.gen_struc(nstruc=nstruc, id_offset=id_offset,
                           init_pos_path='./data/init_POSCARS')
        # ------ molecular crystal
        elif rin.struc_mode == 'mol':
            rsgx.set_mol(mol_file=rin.mol_file, nmol=rin.nmol)
            rsgx.gen_struc_mol(nstruc=nstruc, id_offset=id_offset,
                               init_pos_path='./data/init_POSCARS',
                               timeout_mol=rin.timeout_mol)
        # ------ molecular crystal breaking symmetry
        elif rin.struc_mode == 'mol_bs':
            rsgx.set_mol(mol_file=rin.mol_file, nmol=rin.nmol)
            rsgx.gen_struc_mol_break_sym(nstruc=nstruc,
                                         rot_mol=rin.rot_mol,
                                         nrot=rin.nrot,
                                         id_offset=id_offset,
                                         init_pos_path='./data/init_POSCARS')
        # ------ update
        init_struc_data.update(rsgx.init_struc_data)
    # ---------- Rnd_struc_gen
    else:
        rsg = Rnd_struc_gen(natot=rin.natot, atype=rin.atype, nat=rin.nat,
                            minlen=rin.minlen, maxlen=rin.maxlen,
                            dangle=rin.dangle, mindist=rin.mindist,
                            vol_mu=rin.vol_mu, vol_sigma=rin.vol_sigma,
                            maxcnt=rin.maxcnt, symprec=rin.symprec)
        if rin.spgnum == 0:
            rsg.gen_wo_spg(nstruc=nstruc, id_offset=id_offset,
                           init_pos_path='./data/init_POSCARS')
            init_struc_data.update(rsg.init_struc_data)
        else:
            fwpath = utility.check_fwpath()
            rsg.gen_with_find_wy(nstruc=nstruc, spgnum=rin.spgnum,
                                 id_offset=id_offset,
                                 init_pos_path='./data/init_POSCARS',
                                 fwpath=fwpath)
            init_struc_data.update(rsg.init_struc_data)

    # ---------- output
    print('')    # for blank line
    with open('cryspy.out', 'a') as fout:
        fout.write('Generated structures up to ID {}\n\n'.format(
            len(init_struc_data)-1))


        pkl_data.save_init_struc(init_struc_data)

    return init_struc_data



In [ ]:
class jobs_restart_WorkChain(WorkChain):
    """
    restart_WorkChain
    
    The content of cryspy_in is changed during the step.
    """
    _CRYSPY_IN = "crpspy.in"
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input("cryspy_in", valid_type=(Str,SinglefileData),
                   help='cryspy_in. (temporary implementation)')
        spec.input("stat", valid_type=ConfigparserData,
                   help='ConfigParser data')
        spec.input("init_struc", valid_type=PystructuredictData,
                   help='initial structures')
        spec.outline(
            cls.restart,
        )
        spec.output("init_struc", valid_type=PystructuredictData, help='initial structures')
        spec.output("stat", valid_type=ConfigparserData, help='content of ConfigParser')
  
    def restart(self):
        stat = self.inputs.stat.configparser
        init_struc_data = self.inputs.init_struc.pystructuredict
        stat, init_struc_data = restart(aiida=True, stat=stat, 
                                                       init_struc_data=init_struc_data)
        statdata = ConfigparserData(stat)
        statdata.store()
        self.out('stat', statdata)
        init_struc = PystructuredictData(init_struc_data)
        init_struc.store()
        self.out('init_struc',init_struc )
        

In [ ]:
stat = io_stat.stat_read()
inputs = {"cryspy_in": Str("cpyspy.in"), "stat":ConfigparserData(stat), 
          "init_struc": initial_result["init_struc"]}
restart_results = run(jobs_restart_WorkChain, **inputs)

In [ ]:
restart_results

In [ ]:
stat_node = restart_results["stat"]
stat_node

In [ ]:
stat = stat_node.configparser
stat._sections

# handle_jobs

In [ ]:
def ctrl_next_struc_step1(jobs):
    # ---------- RS
    if rin.algo == 'RS':
        next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- BO
    elif rin.algo == 'BO':
        next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- LAQA
    elif rin.algo == 'LAQA':
        if jobs.laqa_struc[jobs.current_id]:    # vacant list?
            next_struc_data = jobs.laqa_struc[jobs.current_id][-1]
        else:
            next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- EA
    elif rin.algo == 'EA':
        next_struc_data = jobs.init_struc_data[jobs.current_id]
    # ---------- algo is wrong
    else:
        raise ValueError('Error, algo')
        
    return next_struc_data

def ctrl_next_struc_step2(jobs, next_struc_data ):
    # ---------- common part
    # ------ in case there is no initial strucure data
    if next_struc_data is None:
        print('ID {:>6}: initial structure is None'.format(
        jobs.current_id))
        jobs.ctrl_skip()
    # ------ normal initial structure data
    else:
        if False:
            # -- prepare input files for structure optimization
            if rin.kpt_flag:
                print("kpt_flag not supported")
                raise ValueError("kpt_flag not supported")
                jobs.kpt_data = select_code.next_struc(next_struc_data,
                                                   jobs.current_id,
                                                   jobs.work_path,
                                                   jobs.kpt_data)
            else:
                print("select_code.next_struc")
                print(jobs.current_id, jobs.work_path)
                select_code.next_struc(next_struc_data, jobs.current_id,
                                   jobs.work_path)
            
        # -- prepare jobfile
        print("prepare_jobfile")
        # jobs.prepare_jobfile()
        # -- submit
        print("submit_next_struc")
        jobs.submit_next_struc(dry_run=True)
        print('ID {:>6}: submit job, Stage 1'.format(jobs.current_id))
        # -- update status
        # jobs.update_status(operation='submit')

def ctrl_next_struc(jobs):
    next_struc_data = ctrl_next_struc_step1(jobs)

    ctrl_next_struc_step2(jobs,next_struc_data)
    print("must call jobs.update_status(operation='submit')")
    
    
def handle_job_return_dict(jobs):
    print('\n# ---------- job status')
    cwd_dict = {}
    struc_dict = {}
    print("jobs.job_stat", jobs.job_stat )

    for cid in jobs.tmp_running:
        # ---------- set work_path and current_id
        jobs.work_path = './work/{:06}/'.format(cid)
        jobs.current_id = cid
        # ---------- handle job
        if jobs.job_stat[cid] == 'submitted':
            print('ID {:>6}: still queueing or running'.format(cid))
        elif jobs.job_stat[cid] == 'done':
            jobs.ctrl_done()
        elif jobs.job_stat[cid] == 'skip':
            jobs.ctrl_skip()
        elif jobs.job_stat[cid] == 'else':
            raise ValueError('Wrong job_stat in {}. '.format(
                jobs.work_path))
        elif jobs.job_stat[cid] == 'no_file':
            ctrl_next_struc(jobs)
            key = str(cid)
            cwd_dict[key] = os.path.abspath(jobs.work_path)
            struc_dict[key] = jobs.init_struc_data[jobs.current_id]
        else:
            raise ValueError('Unexpected error in {}stat_job'.format(
                jobs.work_path))
        
    return cwd_dict, struc_dict, jobs.tmp_running

def finish_job(jobs):
    for cid in jobs.tmp_running:
        jobs.current_id = cid
        jobs.update_status(operation='submit')
    

In [ ]:
class handle_job_before_submit_WorkChain(WorkChain):
    """
    restart_WorkChain
    
    execute
        jobs = Ctrl_job(stat, init_struc_data)
        jobs.check_job()
        cwd_dict, struc_dict,id_running = handle_job_return_dict(jobs)  
    
    
    The content of cryspy_in is changed during the step.
    """
    _CRYSPY_IN = "crpspy.in"
    @classmethod
    def define(cls, spec):
        super().define(spec)

        spec.input("stat", valid_type=ConfigparserData,
                   help='ConfigParser data')
        spec.input("init_struc", valid_type=PystructuredictData,
                   help='initial structures')
        spec.outline(
            cls.handle_jobs,
        )
        spec.output("struc", valid_type=PystructuredictData, help='structures to optimize')
        spec.output("cwd", valid_type=Dict, help='ConfigParser content')
        spec.output('id_running', valid_type=List, help='id running')
  
    def handle_jobs(self):
        
        stat = self.inputs.stat.configparser
        init_struc_data = self.inputs.init_struc.pystructuredict
        
        os.makedirs("work", exist_ok=True)
        jobs = Ctrl_job(stat, init_struc_data)
        jobs.check_job()
        cwd_dict, struc_dict,id_running = handle_job_return_dict(jobs)  

        if len(cwd_dict)>0:
            cwd_dict = Dict(dict=cwd_dict)
            cwd_dict.store()
            self.out('cwd', cwd_dict)

        if len(struc_dict)>0:
            init_struc = PystructuredictData(struc_dict)
            init_struc.store()
            self.out('struc',init_struc )
        
        id_running=List(list=id_running)
        id_running.store()
        self.out('id_running', id_running)

In [ ]:
inputs = restart_results
handle_job_step1 = run(handle_job_before_submit_WorkChain, **inputs)


In [ ]:
handle_job_step1

In [ ]:
handle_job_step1["id_running"]

In [ ]:
handle_job_step1["cwd"].get_dict()

In [ ]:
handle_job_step1["struc"].get_dict()

In [ ]:
struc_node = handle_job_step1["struc"]
struc_node.pystructuredict

In [ ]:
cwd_node = handle_job_step1["cwd"]
cwd_node.get_dict()

In [ ]:
computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label


In [ ]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes


In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [ ]:
code_lammps_opt

In [ ]:
if len(struc_node.get_dict())>0 and len(cwd_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": struc_node,
             "cwd": cwd_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

In [ ]:
optimize_result

In [ ]:
handle_job_step1["id_running"]

In [ ]:
class submitted_job_status_WorkChain(WorkChain):
    
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input("stat", valid_type=ConfigparserData, help='ConfigParser content')
        spec.input('struc', valid_type=PystructuredictData, help='Structure data')
        spec.input("tmp_running", valid_type=List, help='tmp id running')
        
        spec.outline(
            cls.submitted_job_status,
        )
        spec.output("id_running", valid_type=List,  help='id running')
        spec.output("id_queueing", valid_type=List,  help='id queueing')

    def submitted_job_status(self):
        stat_node = self.inputs.stat
        struc_node = self.inputs.struc
        tmp_running = self.inputs.tmp_running
        jobs = Ctrl_job(stat_node.configparser, struc_node.pystructuredict)
        jobs.tmp_running = tmp_running.get_list()
        finish_job(jobs)
                   
        if len(jobs.id_running)>0:
            id_running = List(list=jobs.id_running)
            print(id_running)
            id_running.store()
            self.out('id_running',id_running)
        
        if len(jobs.id_queueing)>0:
            id_queueing = List(list=jobs.id_queueing)
            id_queueing.store() 
            print(id_queueing)
            self.out('id_queueing', id_queueing)

inputs = {'stat': stat_node, 'struc': struc_node, 
          'tmp_running': handle_job_step1["id_running"]}
submitted_job_status = run(submitted_job_status_WorkChain, **inputs)
submitted_job_status

In [ ]:
if "id_running" in submitted_job_status:
    id_running  =  submitted_job_status["id_running"]
else:
    id_running = []
if "id_queueing" in submitted_job_status:
    id_queueing  =  submitted_job_status["id_queueing"]
else:
    id_queueing = []
id_queueing, id_running

In [ ]:
jobs = Ctrl_job(stat_node.configparser, struc_node.pystructuredict)
print(not (id_queueing or id_running))
if not (id_queueing or id_running):
    jobs.next_sg()
    

# next run

In [ ]:
list(struc_node.get_dict().keys())


In [ ]:
stat, struc_dict = cryspy_restart.restart()
stat._sections

In [ ]:
stat_node

In [ ]:
inputs = {"cryspy_in": Str("cpyspy.in"), "stat":stat_node, 
          "init_struc": struc_node}
restart_results = run(jobs_restart_WorkChain, **inputs)
restart_results

In [ ]:
restart_results["stat"].configparser._sections

In [ ]:
if False:
    stat, struc_dict = cryspy_restart.restart()
    jobs = Ctrl_job(stat, struc_dict)
    jobs.check_job()
    cwd_dict, struc_dict = handle_job_return_dict(jobs)
else:
    inputs = {"stat": stat_node, "init_struc": struc_node}
    handle_job_step1 = run(handle_job_before_submit_WorkChain, **inputs)
    

In [ ]:
handle_job_step1

In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
cwd_dict, struc_dict


In [ ]:
len(struc_dict)


In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)
    

In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
from CrySPY.IO import read_input as rin
from CrySPY.EA import ea_next_gen
import os

'''
Generational change in evolutionary algorithm
'''

import pandas as pd

from CrySPY.EA.ea_child import child_gen
from CrySPY.gen_struc.EA.select_parents import Select_parents
from CrySPY.IO import out_results
from CrySPY.IO import change_input, io_stat, pkl_data
from CrySPY.IO import read_input as rin


def next_gen(stat, init_struc_data, opt_struc_data, rslt_data, ea_id_data):
    # ---------- ea_id_data
    gen, id_queueing, id_running = ea_id_data

    # ---------- out and log
    with open('cryspy.out', 'a') as fout:
        fout.write('# ---------- Evolutionary algorithm\n\n')
        fout.write('# ------ Generation {}\n'.format(gen + 1))
    print('# ---------- Evolutionary algorithm\n')
    print('# ------ Generation {}'.format(gen + 1))

    # ---------- current generation
    c_rslt = rslt_data[rslt_data['Gen'] == gen]
    c_fitness = c_rslt['E_eV_atom'].to_dict()    # {ID: energy, ...}

    # ---------- load ea_data, ea_data is used only in this module
    elite_struc, elite_fitness, ea_info, ea_origin = pkl_data.load_ea_data()

    # ---------- instantiate Seclect_parents class
    print('# -- select parents')
    sp = Select_parents(opt_struc_data, c_fitness, elite_struc, elite_fitness,
                        rin.fit_reverse, rin.n_fittest,
                        rin.emax_ea, rin.emin_ea)
    if rin.slct_func == 'TNM':
        sp.set_tournament(t_size=rin.t_size)
    else:
        sp.set_roulette(a=rin.a_rlt, b=rin.b_rlt)

    # ---------- generate offspring by EA
    print('# -- Generate structures')
    _, eagen = child_gen(sp, init_struc_data)

    # ---------- select elite
    if rin.n_elite > 0:
        print('# -- select elites')
        # ------ init
        all_fitness = rslt_data['E_eV_atom'].to_dict()    # {ID: energy, ..,}
        elite_struc = {}
        elite_fitness = {}
        # ------ Select_parents class also works for selecting elite structures
        se = Select_parents(opt_struc_data, all_fitness, None, None,
                            rin.fit_reverse, rin.n_elite,
                            rin.emax_ea, rin.emin_ea)
        for cid in se.ranking_dedupe:
            print('Structure ID {0:>6} keeps as the elite'.format(cid))
            elite_struc[cid] = opt_struc_data[cid]
            elite_fitness[cid] = all_fitness[cid]
    else:
        elite_struc = None
        elite_fitness = None
    # ---------- out
    with open('cryspy.out', 'a') as fout:
        fout.write('{} structures keeps as the elite\n'.format(rin.n_elite))

    # ---------- new generation
    gen += 1

    # ---------- id_queueing
    id_queueing = [i for i in range(rin.tot_struc, rin.tot_struc + rin.n_pop)]

    # ---------- ea_info
    tmp_info = pd.Series([gen, rin.n_pop, rin.n_crsov, rin.n_perm,
                          rin.n_strain, rin.n_rand, rin.n_elite,
                          rin.crs_lat, rin.slct_func],
                         index=ea_info.columns)
    ea_info = ea_info.append(tmp_info, ignore_index=True)
    # ------ out ea_info
    out_results.out_ea_info(ea_info)

    # ---------- ea_origin
    # ------ EA operation part
    for cid in range(rin.tot_struc, rin.tot_struc + rin.n_pop - rin.n_rand):
        tmp_origin = pd.Series([gen, cid, eagen.operation[cid],
                                eagen.parents[cid]], index=ea_origin.columns)
        ea_origin = ea_origin.append(tmp_origin, ignore_index=True)
    # ------ random part
    for cid in range(rin.tot_struc + rin.n_pop - rin.n_rand,
                     rin.tot_struc + rin.n_pop):
        tmp_origin = pd.Series([gen, cid, 'random', None],
                               index=ea_origin.columns)
        ea_origin = ea_origin.append(tmp_origin, ignore_index=True)
    # ------ elite part
    if rin.n_elite > 0:
        for cid in se.ranking_dedupe:
            tmp_origin = pd.Series([gen, cid, 'elite', 'elite'],
                                   index=ea_origin.columns)
            ea_origin = ea_origin.append(tmp_origin, ignore_index=True)
    # ------ out ea_origin
    out_results.out_ea_origin(ea_origin)

    # ---------- save ea_id_data
    ea_id_data = (gen, id_queueing, id_running)
    pkl_data.save_ea_id(ea_id_data)

    # ---------- save ea_data
    ea_data = (elite_struc, elite_fitness, ea_info, ea_origin)
    pkl_data.save_ea_data(ea_data)

    # ---------- change the value of tot_struc
    config = change_input.config_read()
    change_input.change_basic(config, 'tot_struc', rin.tot_struc + rin.n_pop)
    change_input.write_config(config)
    print('# -- changed cryspy.in')
    print('Changed the value of tot_struc in cryspy.in'
          ' from {} to {}'.format(
              rin.tot_struc, rin.tot_struc + rin.n_pop))

    # ---------- status
    io_stat.set_input_common(stat, 'basic', 'tot_struc', rin.tot_struc + rin.n_pop)
    io_stat.set_common(stat, 'generation', gen)
    io_stat.set_id(stat, 'id_queueing', id_queueing)
    io_stat.write_stat(stat)

    return elite_struc, elite_fitness, ea_info, ea_origin, stat


def next_gen_EA(jobs):
    # ---------- log and out
    with open('cryspy.out', 'a') as fout:
        fout.write('\nDone generation {}\n\n'.format(jobs.gen))
    print('\nDone generation {}\n'.format(jobs.gen))
    # ---------- check point 3
    if rin.stop_chkpt == 3:
        print('\nStop at check point 3: EA is ready\n')
        os.remove('lock_cryspy')
        raise SystemExit()
    # ---------- maxgen_ea
    if 0 < rin.maxgen_ea <= jobs.gen:
        print('\nReached maxgen_ea: {}\n'.format(rin.maxgen_ea))
        os.remove('lock_cryspy')
        raise SystemExit()
    # ---------- EA
    ea_id_data = (jobs.gen, jobs.id_queueing, jobs.id_running)
    elite_struc, elite_fitness, ea_info, ea_origin, stat = \
        ea_next_gen.next_gen(jobs.stat, jobs.init_struc_data,
                             jobs.opt_struc_data, jobs.rslt_data, ea_id_data)

def next_sg(jobs):
    '''
    next selection or generation
    '''
    if rin.algo == 'BO':
        # job.next_select_BO()
        raise ValueError('not supported.')
    if rin.algo == 'LAQA':
        # job.next_select_LAQA()
        raise ValueError('not supported.')
    if rin.algo == 'EA':
        elite_struc, elite_fitness, ea_info, ea_origin, stat = next_gen_EA(jobs)
        return elite_struc, elite_fitness, ea_info, ea_origin, stat 
    

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    next_sg(jobs)
    

# next run

In [ ]:
init_struc_data.keys()

In [ ]:
stat, struc_dict = cryspy_restart.restart()
struc_dict

In [ ]:
jobs = Ctrl_job(stat, struc_dict) # init_struc_dataの更新の必要がある。
jobs.check_job()

In [ ]:
stat["basic"]["tot_struc"]

In [ ]:
cwd_dict, struc_dict = handle_job_return_dict(jobs)

In [ ]:
jobs.init_struc_data

In [ ]:
jobs.id_queueing, jobs.id_running


In [ ]:
cwd_dict, struc_dict


In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)
    

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()
    

# next run


In [ ]:
stat, struc_dict = cryspy_restart.restart()
jobs = Ctrl_job(stat, struc_dict)
jobs.check_job()
cwd_dict, struc_dict = handle_job_return_dict(jobs)


In [ ]:
jobs.id_queueing, jobs.id_running

In [ ]:
cwd_dict, struc_dict

In [ ]:
if len(struc_dict)>0 and len(cwd_dict)>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": Dict(dict=struc_dict),
             "cwd": Dict(dict=cwd_dict),
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    finish_job(jobs)

In [ ]:
print(len(jobs.id_queueing), len(jobs.id_running))
if not (jobs.id_queueing or jobs.id_running):
    jobs.next_sg()

In [ ]:
!verdi graph generate 14891

In [ ]:
!pwd